# Lab. TRIL - Net Zero

Engenharia Computacional para a Emissão Zero no Setor de Óleo e Gás

## Introdução 

Esse é um programa que realiza um processo de leitura, processamento e análise de um conjunto de dados de um reservatório, para encontrar classes de injetividade apresentadas com análise de gráfico.

As funcões a serem desenvolvidas são:
* read_df (recebe inputs csv ou df)
* calc_j (de J para J normalizado)
* proc (KDE e Bayesian Blocks)
* class (encontrar os pontos e classes)
* graf_j (analise dos graficos )

Esse projeto é desenvolvido por:<br>

### Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
import warnings

#Serve para ignorar os 'red warnings' que algumas bibliotecas apontam porque tem novas versoes de implementacao
warnings.filterwarnings('ignore') 

## 1 - Preparação de Files 

Essas funções devem receber arquivos csv (em uma formatação pré-estabelecida) que serem lidos, checados e iniciados.<br>
Os arquivos precisam:
* ter colunas nomeadas (de preferência em ordem alfabética)
* não ter valores NAN

In [3]:
'''função de leitura do arquivo csv para df'''

def read_df(arq):
    #leitura pelo pandas
    df = arq           
    #limpeza (valores NAN)
    if any(df.isna()):
        print("df com valores NAN")
        for i in df:
            df = df.fillna(0)
        print("df corrigido, não mais possui valores NAN")
    
    return df

In [7]:
'''função de checagem de keys em dataframe '''

def checking(df):

    #recebe função de leitura
    df = read_df(df)

    #verificando se falta alguma coluna
    lista_primarias = ['RQI','pressao','pressao_inversa','distancia','permeabilidade','porosidade']
    lista_Js = ['J1', 'J2', 'J3', 'J4', 'J5', 'J6', 'J7', 'J8']  
    
    #verifica se o df possui as keys necessárias
    for i in df.keys(): 
        if lista_primarias.count(i) == 0 and lista_Js.count(i) == 0:
            raise KeyError(f'variavel {i} não identificada')                      
            
        
    return df

## 2 - Calculo dos J's

In [36]:
def normaliza_J (df, J, norm=False):
    
    #recebendo função de checagem
    df = checking(df)
    
    #atribuição        
    for j in J: 
              
        if j == 'J1':
            J1 = df['RQI']
            df['J1'] = J1
            normaliza_(df, j, norm)            
                    
        if j == 'J2':
            J2 = df['RQI']*df['pressao']
            df['J2'] = J2
            normaliza_(df, j, norm)

        if j == 'J3':
            J3 = df['RQI']*df['pressao_inversa']
            df['J3'] = J3
            normaliza_(df, j, norm)                                                                                         
                            
        if j == 'J4':
            #para evitar resultados infinitos
            aux = np.log(df['distancia']) #calc do log(0 ind, 1 inf)
            #mascara
            aux[np.isinf(aux)] = 0 
            aux[np.isnan(aux)] = 0
            aux[aux < 0] = 0

            J4 = df['RQI'] * df['pressao'] * aux      
            df['J4'] = J4
            normaliza_(df, j, norm) 

        if j == 'J5':

            #para evitar resultados infinitos
            aux = np.log(df['distancia']) #calc do log(0 ind, 1 inf)
            #mascara 
            aux[np.isinf(aux)] = 0 
            aux[np.isnan(aux)] = 0
            aux[aux < 0] = 0

            J5 = df['RQI'] * df['pressao_inversa'] * aux
            df['J5'] = J5
            normaliza_(df, j, norm)
        
        if j == 'J6':

            #para evitar resultados infinitos
            aux = np.log(df['distancia']) #calc do log(0 ind, 1 inf)
            #mascara 
            aux[np.isinf(aux)] = 0 
            aux[np.isnan(aux)] = 0
            aux[aux < 0] = 0

            J6 = df['permeabilidade'] * df['porosidade'] * aux
            df['J6'] = J6
            normaliza_(df, j, norm)  

    return df

def normaliza_(df, col, norm):
    if norm == True:   
        df[f'{col}_normalizado'] = (df[col] - min(df[col])) / (max(df[col]) - min(df[col]))
    print(df)

## 3- Processamento Binning <br> (Bayesian Blocks ou KDE)
<br/>
A classe binning é responsável por processar os dados em KDE ou Bayesian Blocks

In [1]:
class binning():

        #método construtor
        def __init__(self) -> None:
            pass
    
        def proc_binning(df, J, binning): #recebe o df e o tipo de binning 

            if binning == 'kde':
                
                '''Bibliotecas'''
                import numpy as np
                from numpy import array, linspace
                from sklearn.neighbors import KernelDensity
                from scipy.misc import electrocardiogram
                from scipy.signal import argrelmin, find_peaks
                from sklearn.model_selection import GridSearchCV, LeaveOneOut
                import scipy.integrate as integrate
                import scipy.special as special
                
                from keras.models import Sequential
                from keras.layers import Dense
                from keras.layers import Dropout
                from keras.wrappers.scikit_learn import KerasClassifier
                from keras.constraints import maxnorm
                from keras.utils import np_utils              
                
                # método de bandwith para kde
                
                """
                # esse metodo retorna 0.0
                for coluna in J:
                    
                    J = df[f'{coluna}'].values
                    band = 1.06*np.std(X) * ((-1/2) ** len(X))
                    print("bandwidth: ", band)
                    #NÃO FUNCIONA dá 0.0 e KeyError: '0.42121'

                
                #encontrar as curvas com bandwidth ideal com validacao cruzada
                #BANDWIDTH ORIGINAL: 0.01
                
                for coluna in J:                   
                    X = df[f'{coluna}'].values[::].reshape(-1, 1)
                
                    band = np.linspace(0.01, 0.05)
                    grid = GridSearchCV(KernelDensity(), {'bandwidth': band},cv=LeaveOneOut())
                    grid.fit(X[:,None].reshape(-1, 1)) 
                    ideal_band = grid.best_params_
                    print(ideal_band)
                    
                    #para o J1 simplificado ele retorna {'bandwidth': 0.01}
                    #para os outros Js faltou memoria para rodar o codigo. 
                """      
                   
                #Calculo KDE   

                for coluna in J:
                    
                    min_i = 0
                    max_i = df[f'{coluna}'].shape[0]#pega a dimensao da coluna 
                    aux = max_i - min_i
                    
                    # faz o reshape para encaixar os dados em uma dimensão apropriada para o calculo 
                    X = df[f'{coluna}'].values[::].reshape(-1, 1)
                    print(X)
                    # calculo da densidade de probabilidade
                    kde = KernelDensity(kernel='gaussian', bandwidth = 0.01).fit(X)
                    print(kde)
                    #retorna os pontos em uma distancia equidistante
                    dist = np.linspace(0, 1.0, aux)
                    print(dist)
                    #calcula a probabilidade logarítmica de cada amostra sob o modelo 
                    #problema de retorno de não array

                    #TRANSFORMAR O RETORNO EM ARRAY

                    #FAÇA o score_samples retornar um array
                    for i in range(len(dist)):
                        log = kde.score_samples(dist[i])
                        print(log)


                    """
                    
                    #Calculo da Integral sobre o score_samples 
                    
                    #reformular a integral pois mesmo com ideal_bandwidth = 0.01,
                    # a integral não está no intervalo correto 
                    import sympy as sp
                    from sympy import init_printing
                    from sympy import integrate, Symbol, dsolve, Function, diff, Eq
                    from sympy import exp, log, sqrt, pi, Integral
                    
                    
                    #integral = 
                    integral = integrate.trapezoid(y=np.exp(log), x=None, dx=dist, axis=-1)
                    
                    # usamos exponencial para deixar log positivo
                    #integral = np.exp(integrate.trapz(log, dist))

                    print(integral)
                    
                    #if 0.99 <= integral <= 1.01:
                    if integral <= 1.01 and integral >= 0.99:
                        print(f'A integral : {integral} é adequeada.')
                    else:
                        print(f'A integral não está no intervalo correto: {integral}')
                        raise KeyError(f'A integral precisa estar entre 0.99 e 1.01')
                    
                    
                    '''Calculo das Particoes'''

                    # calcula os picos
                    peaks = find_peaks(log, height=min(log)) 
                    #Calcula a minima relativa dos dados
                    valleys = argrelmin(log)[0] 
                    
                    #Retorna os indices que classificam o array em ordem crescente
                    ord = np.argsort(np.abs(np.diff(log[valleys])))
                    #inverte a ordem do array 
                    ordValleys_J = np.flip(ord)+1 

                
                #Encontrando Classes 
                # metodo 1
                if len(peaks[0]) == 1:
                    classes = [0, peaks[0], max_i]
                else:   
                    classes = [0, peaks[0][0], peaks[0][1], max_i]
                
                
                
                for coluna in J:

                    X = sorted(df[f'{coluna}'])                           
                    minimo = min(X)
                    classes = [minimo]
                    for i in range(len(peaks)):
                        classes.append(X(peaks[i]))                        
                    classes.append(max(X))
                
                
                #retorno de funcão KDE com dicionario
                resultados_kde = {}
                resultados_kde['log'] = log
                resultados_kde['peaks'] = peaks
                resultados_kde['valleys'] = valleys
                resultados_kde['ordValleys_J'] = ordValleys_J

                return resultados_kde    
                """

            #Calculo Bayesian Blocks

            if binning == 'bb':
                from astropy.stats import bayesian_blocks
                
                resultados_bb = {}

                for coluna in df.columns:
                    if coluna in df.columns[12::]:
                        serie = df.query(f"{coluna} > 0")[coluna]
                        resultados_bb[coluna] = [serie, bayesian_blocks(serie)]

            return resultados_bb
            

In [ ]:
                    '''

                    log = kde.score_samples(dist.reshape(-1,1)) 
                    print(kde.score_samples)
                    print(log)
                    
                    
                    # usamos exponencial para deixar log positivo
                    integral = np.exp(integrate.trapz(log, dist))

                    print(integral)
                    
                    #if 0.99 <= integral <= 1.01:
                    if integral <= 1.01 and integral >= 0.99:
                        print(f'A integral : {integral} é adequeada.')
                    else:
                        print(f'A integral não está no intervalo correto: {integral}')
                        raise KeyError(f'A integral precisa estar entre 0.99 e 1.01')

                    '''

In [ ]:
'''Adição do padding '''

def padding(df):

    padDF = np.pad(df, (0.01, 0.01)) #para a melhor localização de pontos divisores de classe
    return padDF
    

## 4 - Pickle

In [ ]:
def pickle (df, fileout):
    import pickle as pkl  


### Graficos dos Binnings

## 5 - Análise dos Gráficos

In [24]:
class graficos:
    """
    def grafico_kde(df, coluna):
        
        #df deve ser aquele resultante do processamento

        plt.figure(figsize=(20, 12))
        #adaptar para cada J
        plt.title("Vales dos Kernels | J2 normalizado", fontsize=22)
        plt.ylabel("Densidade", fontsize=16)
        plt.xlabel("Posição assumida por J2 norm", fontsize=16)
        plt.grid(True)
        #in a for
        plt.plot(x)
        plt.plot(valleys, x[valleys], 'o')
        plt.plot(valleys[ordValleys_J[0:4]], x[valleys[ordValleys_J[0:4]]], '*', c='green')
        plt.show()

        for i in df:
            print(valleys)

    def grafico_bb(df, bins_bb, coluna):

        #modificar labels para for
        labels = {
            "J1_normalizado" : "RQI",
            "J2_normalizado" : "RQI * Pressao",
            "J3_normalizado" : "RQI * Pressao Inversa",
            "J4_normalizado" : "RQI * Pressao * ln(distancia)",
            "J5_normalizado" : "RQI * Pressao Inversa * ln(distancia)", 
            "J6_normalizado" : "Permeabilidade * Porosidade * ln(distancia)",
            "J7_normalizado" : "ln(Permeabilidade) * ln(Porosidade) * ln(distancia)"
        }
        
        ax = plt.figure(figsize=(20, 12))
        ax = plt.title(f"Histograma '{coluna} = {labels[coluna]}' utilizando blocos bayesianos", fontsize = 24)
        ax = plt.xlabel("X", fontsize = 18)
        ax = plt.ylabel("Y", fontsize = 18)
        ax = plt.hist(df, bins = bins_bb, color='g')
        ax = plt.grid(True)
        plt.savefig(f'./dados/Analise de Js/{coluna}.jpeg', format='jpeg')
        plt.show(ax)
        """

## 6 - Menu

Aqui vamos chamar o pipeline e apresentar opções ao usuário chamando as funções

In [ ]:
print(' TRIL - NetZero '.center(30,'*'))
print(' Bem Vindo '.center(30,'*'))

In [4]:
#arq = str(print("Insira o caminho para o arquivo: ")) #recebemos o csv

#valor J1 já normalizado
arq = pd.read_csv("./dados/Simplificados/J1_reduzido.csv") 
#arq = pd.read_csv("./dados/variaveis.csv")

In [5]:
#chamando a funçao de preparação
read_df(arq)

df com valores NAN
df corrigido, não mais possui valores NAN


,J1
0,0.42121
1,0.41414
2,0.45838
3,0.39962
4,0.34914
...,...
3321,0.00000
3322,0.00000
3323,0.00000
3324,0.00000


In [8]:
checking(arq)

df com valores NAN
df corrigido, não mais possui valores NAN


,J1
0,0.42121
1,0.41414
2,0.45838
3,0.39962
4,0.34914
...,...
3321,0.00000
3322,0.00000
3323,0.00000
3324,0.00000


In [38]:
#J = arq.keys().tolist() #pegando as colunas como lista para transformação
J = ['J1', 'J2', 'J3', 'J4', 'J5', 'J6']
#J = ['J1'] #JÁ NORMALIZADO
arq = normaliza_J(arq, J, norm=True)


df com valores NAN
df corrigido, não mais possui valores NAN
            RQI  pressao_inversa  pressao  porosidade  permeabilidade  \
0      0.112898           4543.9   4608.2    0.062768         0.81143   
1      0.085400           4548.2   4604.2    0.050000         0.36985   
2      0.088106           4552.9   4600.0    0.050000         0.39366   
3      0.086471           4557.9   4595.6    0.050000         0.37919   
4      0.088106           4563.0   4590.9    0.050000         0.39366   
...         ...              ...      ...         ...             ...   
85927  0.064116           4597.3   4560.3    0.112380         0.46856   
85928  0.065263           4602.9   4555.2    0.108470         0.46856   
85929  0.066356           4608.3   4550.4    0.104920         0.46856   
85930  0.039787           4613.5   4545.7    0.107880         0.17321   
85931  0.038431           4618.7   4541.1    0.115630         0.17321   

       distancia        J1  J1_normalizado  
0            0.0 

In [28]:
J = ['J1']
#J = ['J1', 'J2', 'J3', 'J4', 'J5', 'J6']
proc = binning.proc_binning(arq, J, binning='kde')
#proc = binning.proc_binning(arq, J, binning='bb')
proc

{'bandwidth': 0.01}
[[0.42121]
 [0.41414]
 [0.45838]
 ...
 [0.     ]
 [0.     ]
 [0.     ]]
KernelDensity(bandwidth=0.01)
[0.00000000e+00 3.00751880e-04 6.01503759e-04 ... 9.99398496e-01
 9.99699248e-01 1.00000000e+00]
[   1.33875738    1.33830512    1.33694834 ... -216.49720953 -217.11705576
 -217.73780651]


AttributeError: 'function' object has no attribute 'trapezoid'

In [26]:
#com bw = 0.01
#temos que receber o bandwidth por referencia
J = ['J1']
proc = binning.proc_binning(arq, J, binning='kde')

A integral não está no intervalo correto: 0.9521948286229311


KeyError: 'A integral precisa estar entre 0.99 e 1.01'

### opções

In [ ]:
#o df precisa passar pelo calculo dos J's para prosseguir
#o usuário deve ter a opção de escolher quais calculos fazer? sim, quais J' normalizados

In [ ]:
opcao = 0
while opcao != 3:
    print('''   Qual tipo de processamento voce deseja executar?
            digite [1] para Bayesian Blocks
            digite [2] para KDE
            digite [3] para ambos
            digite [4] para voltar ao passo anterior ''')
    opcao = int(input())

    #if opcao == 1:
        # função BB
        #print("Processo de Binning por Bayesian Block completado")
        #print(imagens dos graficos de bayesian blocks para cada J normalizado)
        #o programa pode salvar essas imagens diretamente em um arquivo

    #elif opcao == 2:
        #função KDE
        #print("Processo de Binning por KDE completado")
        #print(imagens dos graficos de KDE para cada J normalizado)
        #o programa pode salvar essas imagens diretamente em um arquivo

    #elif opcao == 3:
        # função BB
        #função KDE
    
    #elif opcao == 4:
        #break #volta do começo

    #else:
        #print("Esse e um caractere invalido. Deseja recomeçar? Digite [0]")
        #opcao = 0

In [ ]:
print("Digite se deseja realizar a classificacao: s/n")
resposta = input()

#if resposta == 's':
    #retornar funcao de classificacao

#if resposta == 'n':
    #print("Fim do Programa")
#else: 
    

In [ ]:
bandwidth = 0.01

Essa função deve receber arquivos csv (em uma formatação pré-estabelecida) que serem lidos, checados e iniciados.<br>
Os arquivos precisam:
* ter colunas nomeadas (de preferência em ordem alfabética)
* não ter valores NAN
* 

In [ ]:
#arq = str(print("Insira o caminho para o arquivo: ")) #recebemos o csv

In [ ]:
#o df precisa passar pelo calculo dos J's para prosseguir
#o usuário deve ter a opção de escolher quais calculos fazer?

In [ ]:
opcao = 0
while opcao != 3:
    print('''   Qual tipo de processamento voce deseja executar?
            digite [1] para Bayesian Blocks
            digite [2] para KDE
            digite [3] para ambos
            digite [4] para voltar ao passo anterior ''')
    opcao = int(input())

    #if opcao == 1:
        # função BB
        #print("Processo de Binning por Bayesian Block completado")
        #print(imagens dos graficos de bayesian blocks para cada J normalizado)
        #o programa pode salvar essas imagens diretamente em um arquivo

    #elif opcao == 2:
        #função KDE
        #print("Processo de Binning por KDE completado")
        #print(imagens dos graficos de KDE para cada J normalizado)
        #o programa pode salvar essas imagens diretamente em um arquivo

    #elif opcao == 3:
        # função BB
        #função KDE
    
    #elif opcao == 4:
        #break #volta do começo

    #else:
        #print("Esse e um caractere invalido. Deseja recomeçar? Digite [0]")
        #opcao = 0

In [ ]:
print("Digite se deseja realizar a classificacao: s/n")
resposta = input()

#if resposta == 's':
    #retornar funcao de classificacao

#if resposta == 'n':
    #print("Fim do Programa")
#else:
    